# Install SemPy

In [ ]:
%pip install semantic-link
%load_ext sempy

# Show datasets in workspace

In [ ]:
import sempy.fabric as fabric
import pyspark.sql.functions as F
df_datasets = fabric.list_datasets()
#mydataset = df_datasets.iloc[0][0]
mydataset = "TheDataModel"
df_datasets



In [ ]:
from sempy.relationships import plot_relationship_metadata

relationships = fabric.list_relationships(mydataset)
plot_relationship_metadata(relationships)

# Run DMV to check Resident columns
Order by Temperature desc

In [ ]:
df_dax = fabric.evaluate_dax(
    mydataset,
    """
    SELECT 
        MEASURE_GROUP_NAME,
        ATTRIBUTE_NAME ,
        DATATYPE ,
        DICTIONARY_SIZE ,
        DICTIONARY_ISPAGEABLE ,
        DICTIONARY_ISRESIDENT ,
        DICTIONARY_TEMPERATURE ,
        DICTIONARY_LAST_ACCESSED
    FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
    ORDER BY [DICTIONARY_TEMPERATURE] DESC
    """)
df_dax.head(20)

# Run some DAX
Sum of Sales, grouped by product color

In [ ]:
myDax = """
    evaluate
        SUMMARIZECOLUMNS(
            DimCalendar[Month] ,
            "Rides" , [Rides]
        )
    """
fabric.evaluate_dax( mydataset,myDax).head(20)

In [ ]:
%%dax TheDataModel

evaluate
SUMMARIZECOLUMNS(
    DimCalendar[Month] ,
    "Rides" , [Rides]
)

# REFRESH Model

In [ ]:
import sempy
import sempy.fabric as fabric
from sempy.fabric._client import DatasetXmlaClient
from sempy.fabric._cache import _get_or_create_workspace_client
sempy.fabric._client._utils._init_analysis_services()

workspace_client = _get_or_create_workspace_client("The Demo")
mydataset = "TheDataModel"
tabular_database = workspace_client.get_dataset(mydataset)
import Microsoft.AnalysisServices.Tabular as Tabular

m = tabular_database.Model
m.RequestRefresh( Tabular.RefreshType.Full)
m.SaveChanges()